![image.png](attachment:image.png)

# Data Science and AI
## Lab 2.2.4: The Google BigQuery UI and API
INSTRUCTIONS:
- Run the cells
- Observe and understand the results
- Answer the questions

## Introduction
The **Google BigQuery UI** provides access to Google's extensive collection of public data sets via an SQL-based query engine.

The **BigQuery API** provides programmatic access to the data sets.

We can use the UI to discover interesting data before writing Python code to access it. Then we can reproduce it in an API request so as to aggregate large amounts of data on Google's infrastructure before pulling the results into our application.

## BigQuery Web UI
- Work through the [Quickstart](https://cloud.google.com/bigquery/docs/quickstarts/quickstart-web-ui).
- You will need to set up a **Google Cloud Platform** account if you do not already have one.
    - **This should not cost anything during the trial period unless you perform a large amount of querying**.
- Afterwards, costs are based on actual resource usage, but most offerings have a free tier.

![image.png](attachment:image.png)

## BigQuery API
You should already have the [**Google Cloud Client Library for Python**](https://cloud.google.com/python/setup) installed.
- Open [Google Cloud Console](https://console.cloud.google.com/home/) and select to create a project.

![image.png](attachment:image.png)

- Under `Getting Started`, select `Enable APIs and get credentials such as keys`.
- In the API table, make sure the BigQuery API is enabled. Page back.

### Authentication
- Go to [Authentication](https://cloud.google.com/docs/authentication/production) page and click the button to create a service account.
![image.png](attachment:image.png)

- Fill out the form, giving the account an appropriate name, and choose `Project Owner` for Account Type
- Click `Create`
![image.png](attachment:image.png)

- The keys will get saved to your computer

![image.png](attachment:image.png)

- Note the location and copy the file path (of the JSON file) to somewhere safe, for future reference.

- You should see a screen like this:

![image.png](attachment:image.png)

- See here for more information:
    - [Managing service account keys](https://cloud.google.com/iam/docs/understanding-service-accounts?&_ga=2.173177830.-495703703.1532572448#managing_service_account_keys)

This is supposed to get implicit key retrieval working:
- Windows
```shell
set GOOGLE_APPLICATION_CREDENTIALS=[PATH]
```
- Linux, MacOS
```shell
export GOOGLE_APPLICATION_CREDENTIALS=[PATH]
```

where `[PATH]` is the full file path of your json key file.

### Using the Python API
Google provides Python libraries for wrapping the Google APIs.

For conda users, these are available on the `conda-forge` channel. There are other Python libraries for Google APIs, possibly adding novel features or ease of use, but Google's is presumably the most current.

Installing the `google-cloud-storage` and `google-cloud-bigquery` libraries should cover all the dependences for this lab.

In [ ]:
!pip install google-cloud-storage
!pip install google-cloud-bigquery
!pip install google-cloud


In [ ]:
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.bigquery import Dataset

If you have managed to get implicit key retrieval working, you can call `Client` with no argument

In [ ]:
storage_client = storage.Client()

If you are not so lucky, you need to invoke a method of the `Client` object that takes the path to your key files as a string argument

In [ ]:
import json

path_key_path = 'auth_gc_bigquery.json'
try:
    auth = json.loads(open(path_auth).read())
except FileNotFoundError as ex:
    print(ex)

In [ ]:
# # For debugging only
# pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(auth)

In [ ]:
key_path = auth['key_path']

This should not throw an error if key retrieval / assignemnt worked

In [ ]:
storage_client = storage.Client.from_service_account_json(key_path)

**Note**: The `storage` object was used in the above example, but there are other objects of interest that have polymorphic `Client` members that are used similarly, such as `bigquery`, which is used below.

If implicit key retrieval is working for you, execute this

In [ ]:
client = bigquery.Client()

If not, execute this

In [ ]:
client = bigquery.Client.from_service_account_json(key_path)

This client is associated with the default project (which was set or defaulted in the BigQuery UI)

In [ ]:
client.project

A BigQuery project contains datasets. Datasets contain tables. To get at the data in a table we need to create a reference that covers this hierarchy; in the `bigquery` library this looks like `project.dataset.table`.  

**Note**: Queries can be performed on prjects and datasets, but most queries are performed on tables.

To explore the public datasets we will start by reassgining our `client` variable using optional `project` parameter (set to `bigquery-public-data`):

In [ ]:
client = bigquery.Client.from_service_account_json(
    key_path,
    project='bigquery-public-data')
print(client.project)

Here is how to get a list of the datasets in the current project

In [ ]:
datasets = list(client.list_datasets())
print(datasets)

That was not helpful. We need to go deeper into the object structure to get at something meaningful. Actually, the `dataset_id` member contains the name attribute of a `dataset` object.

- Write some code to print that name for each member of the list that was created above

The google API objects in the `bigquery` library have their own overloads of the format() function that make them easier to read. 

Below is a function that exploits the `format` method of `project` and `dataset_id`, providing an easy way to list datasets.

In [ ]:
# function for listing datasets in a project:
def printDatasetList(client):
    # only one project can be associated with a client instance
    project = client.project
    datasets = list(client.list_datasets())
    if datasets:
        print('Datasets in project %s:' % project)
        for dataset in datasets:
            print('\t%s' % dataset.dataset_id)
        found = True
    else:
        print('%s project does not contain any datasets.' % project)
        found = False
    return found

In [ ]:
# list datasets in the default project
# assigning to `flag` suppresses printing the return value (normally `True`)
flag = printDatasetList(client)

This list should correspond to what is shown in [public datasets](https://bigquery.cloud.google.com/publicdatasets) under the **bigquery-public-data**.

![image.png](attachment:image.png)

Here is how to create a dataset reference object by assigning a project and a dataset name

In [ ]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id)

If our current project was something other than `bigquery-public-data`, we could still create this reference by specifying the project that contains the dataset

In [ ]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id, project='bigquery-public-data')

**QUESTION**: How can we get the path of the dataset?

In [ ]:
dataset_ref.path

- Explore more of this object's members

In [ ]:
dataset_ref.

Here is a function for listing the tables in a dataset

In [ ]:
# function for listing tables in a dataset:
def printTableList(client, dataset_id):
    project = client.project
    dataset_ref = client.dataset(dataset_id, project=project)
    tables = list(client.list_tables(dataset_ref))
    if tables:
        print('Tables in dataset %s:' % dataset_id)
        for table in tables:
            print('\t%s' % table.table_id)
        found = True
    else:
        print('%s dataset does not contain any tables.' % dataset_id)
        found = False
    return found

Use this function to list the tables in the current dataset

In [ ]:
printTableList(client, dataset_id)

To create a reference to a table within the dataset, we use the `table_id` attribute

In [ ]:
table_id = 'shakespeare'
table_ref = dataset_ref.table(table_id)

Check the name of the table that `table_ref` now points to

In [ ]:
table_ref.table_id

To access the data in the table itself, we use the `get_table()` method

In [ ]:
# API Request
table = client.get_table(table_ref)

**NOTE**: The contents of the table are not actually in our memory after this call! We are working with a Big Data platform, now, and we could easily end up pulling GBs or TBs of data by accident. 

To minimise data bandwidth, memory consumption, and processing time, Big Data platforms employ **lazy evaluation**. This means that no computation or data transfer actually takes place until we **realise** (use) the data. Even if we execute subsequent code that performs calculations on the data, no data flow or computation actually occurs until we request output (e.g. by executing a print to stdout or writing to a file).

**QUESTION**: What kind of object is returned by `client.get_table`?

In [ ]:
type(table)

**QUESTION**: How can we view the design of the table (column names and types?

The name of the boject attribute we need is the same term we learned in the module on databases.

In [ ]:
print(table.schema)

Again, this is messy. If we wanted to refer to the column names and types in code, we might use something like this (which we could then parse into a dict)

In [ ]:
result = ['%s %s' % (schema.name, schema.field_type)
          for schema in table.schema]
print(result)

But if we just want to print them, here is another neat function for that

In [ ]:
# function to print a table schema:
def printTableSchema(aTable):
    schemas = list(aTable.schema)
    if schemas:
        print('Table schema for %s:' % aTable.table_id)
        for aSchema in schemas:
            print('\t%s %s' % (aSchema.name, aSchema.field_type))
        found = True
    else:
        found = False
    return found

- Use this function to print the table schema

In [ ]:
printTableSchema(table)

Now that we know what the columns are, we can write queries. Actually, we construct a query job by assigning an SQL statement to a method of the `client` object

In [ ]:
sql = '''
SELECT COUNT(1)
FROM   `bigquery-public-data.samples.shakespeare`
'''
query_job = client.query(sql)

**QUESTION**: Why does this throw an error?

**ANSWER**: We do not have permission to create queries inside the `bigquery-public-data` project. 

So, what can we do?

In [ ]:
project = 'myreallybigquery'  # <<< your BigQuery project ID here!
client = bigquery.Client.from_service_account_json(key_path, project=project)
query_job = client.query(sql)

If that worked, show what query_job is

In [ ]:
type(query_job)

Once again, due to lazy execution, no actual execution occurs until we request output

In [ ]:
# API request - fetches results
for row in query_job:
    print(row)

And, again, we need to manipulate this to make it neat. Each member of the rowset is a list and we only want to extract the value, which is in the first member

In [ ]:
print(row[0])

So, we now know that this table has 164,656 rows. (We would not want to print it!)

A better coding practice is to write SQL statements that assign names (aliases) to derived values, so we do not forget what the resulting row set contains. Rewite the above SQL statement so that the value returned is aliased a `num_rows`, and assign the QueryJob as above

In [ ]:
sql = '''
SELECT COUNT(1) AS num_rows
FROM   `bigquery-public-data.samples.shakespeare`
'''
query_job = client.query(sql)

Now we could use Python's `assert` statement to build a test into the first code block that operates on the row set

In [ ]:
for row in query_job:  # API request - fetches results
    # Row values can be accessed by field name or index:
    assert row[0] == row.num_rows == row['num_rows']  #: for debugging bad sql
    print(row.num_rows)

The above code checks that the name attribute of the value in `row[0]` is what we expected (i.e. `num_rows`). Also, it shows that we can refer to a field in a row by its object member `num_rows` or by using the same notation we use for Python dictionaries, `['num_rows']`.

- Write, execute, and print the results of a query that fetches 10 rows from the table, each containing the `word`, `word_count` and `corpus` fields

In [ ]:
sql = '''
SELECT word, word_count, corpus
FROM   `bigquery-public-data.samples.shakespeare`
LIMIT  10
'''
query_job = client.query(sql)
# E: , location='US')
# : OK if client.project = 'myreallybigquery'

# print these as above:
for row in query_job:  # API request - fetches results
    # Now have 3 fields to test
    # (Nb. this approach may be overkill for non-production code):
    assert row[0] == row.word == row['word']
    assert row[1] == row.word_count == row['word_count']
    assert row[2] == row.corpus == row['corpus']
    print(row['word'], row['word_count'], row['corpus'])

**NOTE**: Using `assert` regularly is good practice and will make debugging easier, but is probably overkill for non-production code.

Whenever you catch yourself writing a swag of code to do something that seems rudimentary or low-level, there is a very good chance that you do not need to.

A much easier way to handle the above requirement is to use the `to_dataframe()` method of the QueryJob object:

In [ ]:
df = query_job.to_dataframe()
print(df)

Although the above does not use `assert` (which you might still want to include in some test code), you will be able to tell at a glance if something is wrong with the contents of the DataFrame.

### Final Notes
1. Here is a readable way to code long SQL statements

```python
sql = '''
SELECT word, word_count, corpus
FROM   `bigquery-public-data.samples.shakespeare`
LIMIT 10
'''
```

2. If you had an application that needed to modify the tables or datasets in the `bigquery-public-data` is project, you could copy them to our own project, where you would have the permissions to do as you please with the data (subject to Google's terms of use).
3. We are not limited to the datasets that are already in BigQuery. We can upload tables from our computer, and we can pull data in from other online souces. We will cover these tasks in another module.

### Next Steps
If you wish to pick up a few more skills you can check [Create simple app API](https://cloud.google.com/bigquery/create-simple-app-api). As we have already been through the preliminaries, so you can start at `Download the sample code`.

Alternatively, you can take a deeper dive into the API at [BigQuery usage](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).